Author: Justin Braun

Date: 20221228

Purpose: This notebook is designed to generate a CTGAN model on the basis of the training data, which we can use to generate truly anonymous experimental data.
Due to GDPR we cannot make the actual training data public, so the model cannot be replicated. This notebook thus serves to illustrate our work.
New data can be generated by running the cells labelled "NEW DATA".

In [1]:
###IMPORT PACKAGES###
import pandas as pd
import regex as re
import numpy as np
from sdv.tabular import CTGAN
from sdv.tabular import GaussianCopula
from sdv.tabular import CopulaGAN
import time
from datetime import timedelta
from sdv.constraints import Inequality
from sdv.constraints import FixedCombinations
from sdv.constraints import OneHotEncoding
from sdv.constraints import create_custom_constraint
from sdv.sampling import Condition
from sdmetrics.reports.single_table import QualityReport
from sdv import Metadata
from sdmetrics.single_table import NewRowSynthesis
import json
import copy
pd.options.mode.chained_assignment = None  # default='warn'

In [365]:
###LOAD TRAINING DATA###
real_td = pd.read_csv('../data/00_hidden/td_numeric.csv') #USER: specify source dir

In [366]:
###GENERATE METADATA###

features = pd.read_csv('../data/01_raw/20221028_jb_features_type.csv')
colnames = real_td.columns

meta_dic = {
    'fields': {}   
}

for col in colnames:
    my_type = features.loc[features['feature_names_model'] == col,].iloc[0]['type']
    my_subtype = features.loc[features['feature_names_model'] == col,].iloc[0]['subtype']
    if my_type == 'numerical':
        meta_dic['fields'][col] = {'type':my_type,
                                  'subtype':my_subtype}
        continue
    meta_dic['fields'][col] = {'type':my_type}
    
meta_path = '../data/01_raw/my_metadata_file.json' #USER: specify filepath to save meta data table
with open(meta_path, 'w') as f:
    json.dump(meta_dic, f)


In [367]:
###DEFINE DIFFERENT LISTS OF VARS TO GENERATE TEST/TOY DATA###
districts_vars = ['adres_recentste_wijk_charlois', 'adres_recentste_wijk_delfshaven', 'adres_recentste_wijk_feijenoord',
                   'adres_recentste_wijk_ijsselmonde', 'adres_recentste_wijk_kralingen_c', 'adres_recentste_wijk_noord',
                   'adres_recentste_wijk_other', 'adres_recentste_wijk_prins_alexa', 'adres_recentste_wijk_stadscentru']
neighborhoods_vars = ['adres_recentste_buurt_groot_ijsselmonde', 'adres_recentste_buurt_nieuwe_westen', 'adres_recentste_buurt_other',
                   'adres_recentste_buurt_oude_noorden', 'adres_recentste_buurt_vreewijk']
reading_vars = ['persoonlijke_eigenschappen_nl_lezen3', 'persoonlijke_eigenschappen_nl_lezen4']
writing_vars = ['persoonlijke_eigenschappen_nl_schrijven0', 'persoonlijke_eigenschappen_nl_schrijven1', 'persoonlijke_eigenschappen_nl_schrijven2',
                'persoonlijke_eigenschappen_nl_schrijven3', 'persoonlijke_eigenschappen_nl_schrijvenfalse']
speaking_vars = ['persoonlijke_eigenschappen_nl_spreken1',  'persoonlijke_eigenschappen_nl_spreken2',
                 'persoonlijke_eigenschappen_nl_spreken3']
contacten_vars = ['contacten_onderwerp__arbeids_motivatie','contacten_onderwerp_boolean__arbeids_motivatie',
                 'contacten_onderwerp__pre__intake','contacten_onderwerp_boolean__pre__intake',
                 'contacten_onderwerp__werk_intake','contacten_onderwerp_boolean__werk_intake',
                 'contacten_onderwerp_beoordelen_taaleis','contacten_onderwerp_boolean_beoordelen_taaleis',
                 'contacten_onderwerp_contact_derden','contacten_onderwerp_boolean_contact_derden',
                 'contacten_onderwerp_contact_met_aanbieder','contacten_onderwerp_boolean_contact_met_aanbieder',
                 'contacten_onderwerp_diagnosegesprek','contacten_onderwerp_boolean_diagnosegesprek',
                 'contacten_onderwerp_documenten__innemen_','contacten_onderwerp_boolean_documenten__innemen_',
                 'contacten_onderwerp_documenttype__cv_','contacten_onderwerp_boolean_documenttype__cv_',
                 'contacten_onderwerp_documenttype__overeenkomst_','contacten_onderwerp_boolean_documenttype__overeenkomst_',
                 'contacten_onderwerp_financiële_situatie','contacten_onderwerp_boolean_financiële_situatie',
                 'contacten_onderwerp_groepsbijeenkomst','contacten_onderwerp_boolean_groepsbijeenkomst',
                 'contacten_onderwerp_inkomen','contacten_onderwerp_boolean_inkomen',
                 'contacten_onderwerp_maatregel_overweging','contacten_onderwerp_boolean_maatregel_overweging',
                 'contacten_onderwerp_matching','contacten_onderwerp_boolean_matching',
                 'contacten_onderwerp_mutatie','contacten_onderwerp_boolean_mutatie',
                 'contacten_onderwerp_no_show','contacten_onderwerp_boolean_no_show',
                 'contacten_onderwerp_overige','contacten_onderwerp_boolean_overige',
                 'contacten_onderwerp_overleg_met_inkomen','contacten_onderwerp_boolean_overleg_met_inkomen',
                 'contacten_onderwerp_scholing','contacten_onderwerp_boolean_scholing',
                 'contacten_onderwerp_terugbelverzoek','contacten_onderwerp_boolean_terugbelverzoek',
                 'contacten_onderwerp_traject','contacten_onderwerp_boolean_traject',
                 'contacten_onderwerp_uitnodiging','contacten_onderwerp_boolean_uitnodiging',
                 'contacten_onderwerp_ziek__of_afmelding','contacten_onderwerp_boolean_ziek__of_afmelding',
                 'contacten_onderwerp_zorg','contacten_onderwerp_boolean_zorg']
ontheffing_vars = ['ontheffing_reden_hist_medische_gronden','ontheffing_reden_hist_other', 
                   'ontheffing_reden_hist_sociale_gronden',
                   'ontheffing_reden_hist_tijdelijke_ontheffing_arbeidsverpl__en_tegenprestatie',
                   'ontheffing_reden_hist_tijdelijke_ontheffing_arbeidsverplichtingen',
                   'ontheffing_reden_hist_vanwege_uw_sociaal_maatschappelijke_situatie',
                   'ontheffing_dagen_hist_vanwege_uw_medische_omstandigheden', 
                   'ontheffing_dagen_hist_mean']
typering_vars = ['typering_indicatie_geheime_gegevens', 'typering_other',
                 'typering_transport__logistiek___tuinbouw', 'typering_zorg__schoonmaak___welzijn',
                 'typering_aantal', 'typering_ind', 'typering_hist_inburgeringsbehoeftig', 
                 'typering_hist_sector_zorg', 'typering_dagen_som']
constraint_vars = ['adres_recentste_plaats_other', 'adres_recentste_plaats_rotterdam',
                  'adres_recentst_onderdeel_rdam', 'afspraak_laatstejaar_aantal_woorden',
                  'afspraak_aantal_woorden', 'afspraak_laatstejaar_resultaat_ingevuld_uniek',
                  'afspraak_resultaat_ingevuld_uniek', 'beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden',
                  'beschikbaarheid_huidig_bekend', 'beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden',
                   'beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie', 'beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden',
                  'beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie', 'relatie_kind_heeft_kinderen',
                  'relatie_kind_huidige_aantal', 'relatie_partner_huidige_partner___partner__gehuwd_',
                  'relatie_partner_aantal_partner___partner__gehuwd_', 'pla_ondertekeningen_actueel',
                  'pla_ondertekeningen_historie', 'ontheffing_hist_ind', 
                   'typering_hist_ind', 'typering_hist_aantal', 'persoonlijke_eigenschappen_spreektaal']

In [368]:
###CREATE TOY/TEST DATA###
#USER: the cell can be commented out later on when we have trained the real model
keep_cols = districts_vars + neighborhoods_vars + constraint_vars  #columns to be included in the test
td_sample = real_td.sample(frac=0.1, replace=False, random_state=1) #10% sample
td_sample = td_sample.loc[:, keep_cols]
print(td_sample.shape)

(1264, 37)


In [369]:
###CUSTOM CONSTRAINT: ZERO OR ONE HOT ENCODED###
#is valid when at 1 or 0 of the columns are coded as zero
def zero_one_hot_encoding_is_valid(column_names, data):
    temp=data.loc[:, column_names]
    is_zero_or_one_hot_encoded = (temp.sum(axis=1) <= 1)
    return is_zero_or_one_hot_encoded

ZeroOneHotEncoding = create_custom_constraint(
    is_valid_fn=zero_one_hot_encoding_is_valid
)

#test function
data = [[0,0],[0,1],[1,0],[1,1],[1,2],[2,1],[2,2]]
test_df = pd.DataFrame(data, columns=['refcol', 'col1'])
test_fn = zero_one_hot_encoding_is_valid(['col1', 'refcol'], test_df)
print(test_fn)

0     True
1     True
2     True
3    False
4    False
5    False
6    False
dtype: bool


In [370]:
###CUSTOM CONSTRAINT: ZERO OR INEQUALITY###
#Is valid when all columns are zero or when ref column is smaller than all columns in column name
def zero_or_ge_is_valid(column_names, data, ref_column):
    ret_list = [True for i in range(len(data.index))]
    for col in column_names:
        temp_list = (data[ref_column] >= data[col])
        ret_list = temp_list & ret_list
    return ret_list

ZeroOrGreater = create_custom_constraint(
    is_valid_fn=zero_or_ge_is_valid
)

#test function
data = [[0,0],[0,1],[1,0],[1,1],[1,2],[2,1],[2,2]]
test_df = pd.DataFrame(data, columns=['refcol', 'col1'])
test_fn = zero_or_ge_is_valid(['col1'], test_df, 'refcol')
print(test_fn)


0     True
1    False
2     True
3     True
4    False
5     True
6     True
dtype: bool


In [371]:
###CUSTOM CONSTRAINT: ZEROs MUST MATCH###
#is valid if ref_column is zero and all columns in column_names are also zero
#if ref_column is not zero, other columns can vary
def zero_must_match_is_valid(column_names, data, ref_column):
    ret_list = [True for i in range(len(data.index))]
    for col in column_names:
        temp_list = (((data[ref_column] == 0) & (data[col] == 0)) | (data[ref_column] > 0))
        ret_list = temp_list & ret_list
    return ret_list

ZeroMustMatch = create_custom_constraint(
    is_valid_fn=zero_must_match_is_valid
)

#test function
data = [[0,0],[0,1],[1,0],[1,1],[1,2],[2,1],[2,2]]
test_df = pd.DataFrame(data, columns=['refcol', 'col1'])
test_fn = zero_must_match_is_valid(['col1'], test_df, 'refcol')
print(test_fn)


0     True
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool


In [372]:
###TEST IF REAL TD CONFORMS TO BUSINESS RULES###
#look at https://docs.google.com/document/d/1XeXIFv1rUh3NQEdlgwl8I0k_drGL_hbUQJ18eJy0FkI/edit# 
# for complete list of business rules

test_districts = zero_one_hot_encoding_is_valid(districts_vars, real_td)
print('District check')
print(test_districts.value_counts(), '\n')

test_neighbordhoods = zero_one_hot_encoding_is_valid(neighborhoods_vars, real_td)
print('Neighborhood check')
print(test_neighbordhoods.value_counts(), '\n')

test_plaats = zero_one_hot_encoding_is_valid(['adres_recentste_plaats_other','adres_recentste_plaats_rotterdam'], real_td)
print('Plaats check')
print(test_plaats.value_counts(), '\n')

district_neighborhood_plaats = districts_vars + neighborhoods_vars + ['adres_recentste_plaats_rotterdam']
district_neighborhood_plaats = list(set(district_neighborhood_plaats) - set(['adres_recentste_wijk_delfshaven', 'adres_recentste_wijk_other', 'adres_recentste_buurt_other']))
test_onderdeel = zero_or_ge_is_valid(district_neighborhood_plaats, real_td, 'adres_recentst_onderdeel_rdam')
print('Onderdeel check')
print(test_onderdeel.value_counts(), '\n')

district_neighborhood_plaats.remove('adres_recentste_plaats_rotterdam')
district_neighborhood = district_neighborhood_plaats
test_plaats = zero_or_ge_is_valid(district_neighborhood, real_td, 'adres_recentste_plaats_rotterdam')
print('Plaats check')
print(test_plaats.value_counts(), '\n')

district_neighborhood_matches = {'adres_recentste_wijk_noord':'adres_recentste_buurt_oude_noorden',
                                'adres_recentste_wijk_feijenoord':'adres_recentste_buurt_vreewijk',
                                'adres_recentste_wijk_ijsselmonde':'adres_recentste_buurt_groot_ijsselmonde',
                                'adres_recentste_wijk_delfshaven':'adres_recentste_buurt_nieuwe_westen'}
for key, value in district_neighborhood_matches.items():
    test_district_neighborhood = zero_or_ge_is_valid([value], real_td, key)
    print(key, 'check')
    print(test_district_neighborhood.value_counts(), '\n')

other_district_test = zero_must_match_is_valid(['adres_recentste_wijk_other'], real_td, 'adres_recentste_buurt_other')
print('Other district test')
print(other_district_test.value_counts(), '\n')

test_reading = zero_one_hot_encoding_is_valid(reading_vars, real_td)
print('Reading check')
print(test_reading.value_counts(), '\n')

test_writing = zero_one_hot_encoding_is_valid(writing_vars, real_td)
print('Writing check')
print(test_writing.value_counts(), '\n')

test_speaking = zero_one_hot_encoding_is_valid(speaking_vars, real_td)
print('Speaking check')
print(test_speaking.value_counts(), '\n')

test_aantal_worden = zero_or_ge_is_valid(['afspraak_laatstejaar_aantal_woorden'], real_td, 'afspraak_aantal_woorden')
print('Aantal worden check')
print(test_aantal_worden.value_counts(), '\n')

test_ingelvuld_uniek = zero_or_ge_is_valid(['afspraak_laatstejaar_resultaat_ingevuld_uniek'], real_td, 'afspraak_resultaat_ingevuld_uniek')
print('Ingelvuld uniek check')
print(test_ingelvuld_uniek.value_counts(), '\n')

test_beschikbaarheid_huidig = zero_or_ge_is_valid(['beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden'], real_td, 'beschikbaarheid_huidig_bekend')
print('Beschikbaarheid huidig check')
print(test_beschikbaarheid_huidig.value_counts(), '\n')

test_beschikbaarheid_medische = zero_or_ge_is_valid(['beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden', 'beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden'], real_td, 'beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden')
print('Beschikbaarheid medische check')
print(test_beschikbaarheid_medische.value_counts(), '\n')

test_beschikbaarheid_sociaal = zero_or_ge_is_valid(['beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie'], real_td, 'beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie')
print('Beschikbaarheid sociaal check')
print(test_beschikbaarheid_sociaal.value_counts(), '\n')

test_beschikbaarheid_recent = zero_one_hot_encoding_is_valid(['beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden', 
                                                             'beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie'], real_td)
print('Beschikbaarheid recent check')
print(test_beschikbaarheid_recent.value_counts(), '\n')

contacten_matches = {'contacten_onderwerp__arbeids_motivatie':'contacten_onderwerp_boolean__arbeids_motivatie',
                 'contacten_onderwerp__pre__intake':'contacten_onderwerp_boolean__pre__intake',
                 'contacten_onderwerp__werk_intake':'contacten_onderwerp_boolean__werk_intake',
                 'contacten_onderwerp_beoordelen_taaleis':'contacten_onderwerp_boolean_beoordelen_taaleis',
                 'contacten_onderwerp_contact_derden':'contacten_onderwerp_boolean_contact_derden',
                 'contacten_onderwerp_contact_met_aanbieder':'contacten_onderwerp_boolean_contact_met_aanbieder',
                 'contacten_onderwerp_diagnosegesprek':'contacten_onderwerp_boolean_diagnosegesprek',
                 'contacten_onderwerp_documenten__innemen_':'contacten_onderwerp_boolean_documenten__innemen_',
                 'contacten_onderwerp_documenttype__cv_':'contacten_onderwerp_boolean_documenttype__cv_',
                 'contacten_onderwerp_documenttype__overeenkomst_':'contacten_onderwerp_boolean_documenttype__overeenkomst_',
                 'contacten_onderwerp_financiële_situatie':'contacten_onderwerp_boolean_financiële_situatie',
                 'contacten_onderwerp_groepsbijeenkomst':'contacten_onderwerp_boolean_groepsbijeenkomst',
                 'contacten_onderwerp_inkomen':'contacten_onderwerp_boolean_inkomen',
                 'contacten_onderwerp_maatregel_overweging':'contacten_onderwerp_boolean_maatregel_overweging',
                 'contacten_onderwerp_matching':'contacten_onderwerp_boolean_matching',
                 'contacten_onderwerp_mutatie':'contacten_onderwerp_boolean_mutatie',
                 'contacten_onderwerp_no_show':'contacten_onderwerp_boolean_no_show',
                 'contacten_onderwerp_overige':'contacten_onderwerp_boolean_overige',
                 'contacten_onderwerp_overleg_met_inkomen':'contacten_onderwerp_boolean_overleg_met_inkomen',
                 'contacten_onderwerp_scholing':'contacten_onderwerp_boolean_scholing',
                 'contacten_onderwerp_terugbelverzoek':'contacten_onderwerp_boolean_terugbelverzoek',
                 'contacten_onderwerp_traject':'contacten_onderwerp_boolean_traject',
                 'contacten_onderwerp_uitnodiging':'contacten_onderwerp_boolean_uitnodiging',
                 'contacten_onderwerp_ziek__of_afmelding':'contacten_onderwerp_boolean_ziek__of_afmelding',
                 'contacten_onderwerp_zorg':'contacten_onderwerp_boolean_zorg'}
for key, value in contacten_matches.items():
    test_contacten = zero_or_ge_is_valid([value], real_td, key)
    print(key, 'check')
    print(test_contacten.value_counts(), '\n')
    
test_kind = zero_or_ge_is_valid(['relatie_kind_heeft_kinderen'], real_td, 'relatie_kind_huidige_aantal')
print('Kind check')
print(test_kind.value_counts(), '\n')

test_married = zero_or_ge_is_valid(['relatie_partner_huidige_partner___partner__gehuwd_'], real_td, 'relatie_partner_aantal_partner___partner__gehuwd_')
print('Married check')
print(test_married.value_counts(), '\n')

test_pla_ondertekeningen = zero_or_ge_is_valid(['pla_ondertekeningen_actueel'], real_td, 'pla_ondertekeningen_historie')
print('Pla ondertekeningen check')
print(test_pla_ondertekeningen.value_counts(), '\n')

ontheffing_test = zero_must_match_is_valid(ontheffing_vars, real_td, 'ontheffing_hist_ind')
print('Ontheffing check')
print(ontheffing_test.value_counts(), '\n')

typering_test = zero_must_match_is_valid(typering_vars, real_td, 'typering_hist_ind')
print('Typering check')
print(typering_test.value_counts(), '\n')

typering_hist_test = zero_or_ge_is_valid(['typering_hist_ind'], real_td, 'typering_hist_aantal')
print('Typering History test')
print(typering_hist_test.value_counts(), '\n')

District check
True    12645
dtype: int64 

Neighborhood check
True    12645
dtype: int64 

Plaats check
True    12645
dtype: int64 

Onderdeel check
True    12645
dtype: int64 

Plaats check
True    12645
dtype: int64 

adres_recentste_wijk_noord check
True    12645
dtype: int64 

adres_recentste_wijk_feijenoord check
True    12645
dtype: int64 

adres_recentste_wijk_ijsselmonde check
True    12645
dtype: int64 

adres_recentste_wijk_delfshaven check
True    12645
dtype: int64 

Other district test
True    12645
dtype: int64 

Reading check
True    12645
dtype: int64 

Writing check
True    12645
dtype: int64 

Speaking check
True    12645
dtype: int64 

Aantal worden check
True    12645
dtype: int64 

Ingelvuld uniek check
True    12645
dtype: int64 

Beschikbaarheid huidig check
True    12645
dtype: int64 

Beschikbaarheid medische check
True    12645
dtype: int64 

Beschikbaarheid sociaal check
True    12645
dtype: int64 

Beschikbaarheid recent check
True    12645
dtype: int64 

c

In [373]:
###DEFINE constraints###
#See cell above for business rules spelled out
district_constraint = ZeroOneHotEncoding(
    column_names = districts_vars
)

neighborhood_constraint = ZeroOneHotEncoding(
    column_names = neighborhoods_vars
)

plaats_constraint = ZeroOneHotEncoding(
    column_names=['adres_recentste_plaats_other','adres_recentste_plaats_rotterdam']
)

district_neighborhood_plaats = districts_vars + neighborhoods_vars + ['adres_recentste_plaats_rotterdam']
district_neighborhood_plaats = list(set(district_neighborhood_plaats) - set(['adres_recentste_wijk_delfshaven', 'adres_recentste_wijk_other', 'adres_recentste_buurt_other']))
onderdeel_constraint = ZeroOrGreater(
    column_names = district_neighborhood_plaats,
    ref_column = 'adres_recentst_onderdeel_rdam'
)


district_neighborhood_plaats.remove('adres_recentste_plaats_rotterdam')
district_neighborhood = district_neighborhood_plaats
district_plaats_constraint = ZeroOrGreater(
    column_names = district_neighborhood,
    ref_column = 'adres_recentste_plaats_rotterdam'
)

district_neighborhood_match_constraints = []
for key, value in district_neighborhood_matches.items():
    cur = ZeroOrGreater(
        column_names = [value],
        ref_column = key
    )
    district_neighborhood_match_constraints.append(cur)
    
other_district_constraint = ZeroOrGreater(
    column_names = ['adres_recentste_wijk_other'],
    ref_column = 'adres_recentste_buurt_other'
)

reading_constraint = ZeroOneHotEncoding(
    column_names = reading_vars
)

writing_constraint = ZeroOneHotEncoding(
    column_names = writing_vars
)

speaking_constraint = ZeroOneHotEncoding(
    column_names = speaking_vars
)

aantal_worden_constraint = ZeroOrGreater(
    column_names = ['afspraak_laatstejaar_aantal_woorden'],
    ref_column = 'afspraak_aantal_woorden'
)

ingelvuld_uniek_constraint = ZeroOrGreater(
    column_names = ['afspraak_laatstejaar_resultaat_ingevuld_uniek'],
    ref_column = 'afspraak_resultaat_ingevuld_uniek'
)

beschikbaarheid_huidig_constraint = ZeroOrGreater(
    column_names = ['beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden'],
    ref_column = 'beschikbaarheid_huidig_bekend'
)

beschikbaarheid_medische_constraint = ZeroOrGreater(
    column_names = ['beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden', 
                    'beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden'],
    ref_column = 'beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden'
)

beschikbaarheid_sociaal_constraint = ZeroOrGreater(
    column_names = ['beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie'],
    ref_column = 'beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie'
)

beschikbaarheid_recent_constraint = ZeroOneHotEncoding(
    column_names = ['beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden', 
                    'beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie']
)

contacten_constraints = []
for key, value in contacten_matches.items():
    cur = ZeroOrGreater(
        column_names = [value],
        ref_column = key
    )
    contacten_constraints.append(cur)

    
has_children_constraint = ZeroOrGreater(
    column_names = ['relatie_kind_heeft_kinderen'],
    ref_column = 'relatie_kind_huidige_aantal'
)

married_constraint = ZeroOrGreater(
    column_names = ['relatie_partner_huidige_partner___partner__gehuwd_'],
    ref_column = 'relatie_partner_aantal_partner___partner__gehuwd_'
)

pla_ondertekeningen_constraint = ZeroOrGreater(
    column_names = ['pla_ondertekeningen_actueel'],
    ref_column = 'pla_ondertekeningen_historie'
)

ontheffing_constraint = ZeroMustMatch(
    column_names = ontheffing_vars,
    ref_column = 'ontheffing_hist_ind'
)

typering_constraint = ZeroMustMatch(
    column_names = typering_vars,
    ref_column = 'typering_hist_ind'
)

typering_hist_constraint = ZeroOrGreater(
    column_names = ['typering_hist_ind'],
    ref_column = 'typering_hist_aantal'
)


constraints = [district_constraint, neighborhood_constraint,
              plaats_constraint, onderdeel_constraint, district_plaats_constraint,
              other_district_constraint, reading_constraint,
              writing_constraint, speaking_constraint,
              aantal_worden_constraint, ingelvuld_uniek_constraint,
              beschikbaarheid_huidig_constraint, beschikbaarheid_medische_constraint,
              beschikbaarheid_recent_constraint, beschikbaarheid_sociaal_constraint,
              has_children_constraint, married_constraint,
              pla_ondertekeningen_constraint, ontheffing_constraint,
              typering_constraint, typering_hist_constraint
              ]

constraints = constraints + district_neighborhood_match_constraints + contacten_constraints

# constraints = [district_constraint, neighborhood_constraint, ingelvuld_uniek_constraint, aantal_worden_constraint,
#               beschikbaarheid_huidig_constraint, beschikbaarheid_medische_constraint, beschikbaarheid_recent_constraint,
#               beschikbaarheid_sociaal_constraint]

In [374]:
def adapt_meta_dic(td_sample, meta_dic):
    meta_dic_test = copy.deepcopy(meta_dic)
    my_cols = td_sample.columns
    for key in meta_dic['fields']:
        if key not in my_cols:
            meta_dic_test['fields'].pop(key)
    return meta_dic_test

In [375]:
# meta_dic_sample = adapt_meta_dic(td_sample, meta_dic)
###LOAD MODEL###
model = GaussianCopula(
    constraints=constraints,
    field_types = meta_dic['fields']
)

In [376]:
###FIT MODEL##
start_time = time.monotonic()
model.fit(real_td) #USER: specify the data object to train the model on, usually 'real_td'
end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))

/opt/miniconda3/lib/python3.8/site-packages/copulas/univariate/truncated_gaussian.py:45: RuntimeWarning: divide by zero encountered in double_scalars
  a = (self.min - loc) / scale
/opt/miniconda3/lib/python3.8/site-packages/copulas/univariate/truncated_gaussian.py:46: RuntimeWarning: invalid value encountered in double_scalars
  b = (self.max - loc) / scale
/opt/miniconda3/lib/python3.8/site-packages/copulas/univariate/truncated_gaussian.py:45: RuntimeWarning: invalid value encountered in double_scalars
  a = (self.min - loc) / scale
/opt/miniconda3/lib/python3.8/site-packages/copulas/univariate/truncated_gaussian.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  b = (self.max - loc) / scale


0:00:13.281736


In [378]:
#NEW DATA#


###SAVE MODEL FILE###
path = '../data/01_raw/GaussianCopula-generator.pkl' #USER: specify filepath to save the model
model.save(path)

In [5]:
#NEW DATA#


###SAMPLE FROM NEW MODEL###
#real = real_td #USER: specify the data object that the model was trained on, usually 'real_td'
# model = CTGAN.load('../data/01_raw/CTGAN-generator.pkl')
model = GaussianCopula.load('../data/01_raw/GaussianCopula-generator.pkl')
synth_data = model.sample(num_rows=12707, randomize_samples=1)
print(synth_data.shape)
#synth_data.to_csv('../data/01_raw/synth_data.csv', index=False)

Sampling rows: 100%|██████████████████████| 12707/12707 [03:00<00:00, 70.36it/s]

(12707, 315)


In [380]:
###EVALUATE MODEL PERFORMANCE###
synth = synth_data #USER: change this to the synthetic data generated by your model
real = real_td #USER: change this to the real data you want to evaluate the model against
# meta_dic_sample = adapt_meta_dic(td_sample, meta_dic)

report = QualityReport()
report.generate(real, synth, meta_dic) #generates headline performance score

Creating report: 100%|████████████████████████████| 4/4 [04:42<00:00, 70.71s/it]



Overall Quality Score: 85.58%

Properties:
Column Shapes: 81.69%
Column Pair Trends: 89.47%


In [381]:
###TEST IF REAL TD CONFORMS TO BUSINESS RULES###
#look at https://docs.google.com/document/d/1XeXIFv1rUh3NQEdlgwl8I0k_drGL_hbUQJ18eJy0FkI/edit# 
# for complete list of business rules

test_districts = zero_one_hot_encoding_is_valid(districts_vars, synth_data)
print('District check')
print(test_districts.value_counts(), '\n')

test_neighbordhoods = zero_one_hot_encoding_is_valid(neighborhoods_vars, synth_data)
print('Neighborhood check')
print(test_neighbordhoods.value_counts(), '\n')

test_plaats = zero_one_hot_encoding_is_valid(['adres_recentste_plaats_other','adres_recentste_plaats_rotterdam'], synth_data)
print('Plaats check')
print(test_plaats.value_counts(), '\n')

district_neighborhood_plaats = districts_vars + neighborhoods_vars + ['adres_recentste_plaats_rotterdam']
district_neighborhood_plaats = list(set(district_neighborhood_plaats) - set(['adres_recentste_wijk_delfshaven', 'adres_recentste_wijk_other', 'adres_recentste_buurt_other']))
test_onderdeel = zero_or_ge_is_valid(district_neighborhood_plaats, synth_data, 'adres_recentst_onderdeel_rdam')
print('Onderdeel check')
print(test_onderdeel.value_counts(), '\n')

district_neighborhood_plaats.remove('adres_recentste_plaats_rotterdam')
district_neighborhood = district_neighborhood_plaats
test_plaats = zero_or_ge_is_valid(district_neighborhood, synth_data, 'adres_recentste_plaats_rotterdam')
print('Plaats check')
print(test_plaats.value_counts(), '\n')

district_neighborhood_matches = {'adres_recentste_wijk_noord':'adres_recentste_buurt_oude_noorden',
                                'adres_recentste_wijk_feijenoord':'adres_recentste_buurt_vreewijk',
                                'adres_recentste_wijk_ijsselmonde':'adres_recentste_buurt_groot_ijsselmonde',
                                'adres_recentste_wijk_delfshaven':'adres_recentste_buurt_nieuwe_westen'}
for key, value in district_neighborhood_matches.items():
    test_district_neighborhood = zero_or_ge_is_valid([value], synth_data, key)
    print(key, 'check')
    print(test_district_neighborhood.value_counts(), '\n')

other_district_test = zero_must_match_is_valid(['adres_recentste_wijk_other'], synth_data, 'adres_recentste_buurt_other')
print('Other district test')
print(other_district_test.value_counts(), '\n')

test_reading = zero_one_hot_encoding_is_valid(reading_vars, synth_data)
print('Reading check')
print(test_reading.value_counts(), '\n')

test_writing = zero_one_hot_encoding_is_valid(writing_vars, synth_data)
print('Writing check')
print(test_writing.value_counts(), '\n')

test_speaking = zero_one_hot_encoding_is_valid(speaking_vars, synth_data)
print('Speaking check')
print(test_speaking.value_counts(), '\n')

test_aantal_worden = zero_or_ge_is_valid(['afspraak_laatstejaar_aantal_woorden'], synth_data, 'afspraak_aantal_woorden')
print('Aantal worden check')
print(test_aantal_worden.value_counts(), '\n')

test_ingelvuld_uniek = zero_or_ge_is_valid(['afspraak_laatstejaar_resultaat_ingevuld_uniek'], synth_data, 'afspraak_resultaat_ingevuld_uniek')
print('Ingelvuld uniek check')
print(test_ingelvuld_uniek.value_counts(), '\n')

test_beschikbaarheid_huidig = zero_or_ge_is_valid(['beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden'], synth_data, 'beschikbaarheid_huidig_bekend')
print('Beschikbaarheid huidig check')
print(test_beschikbaarheid_huidig.value_counts(), '\n')

test_beschikbaarheid_medische = zero_or_ge_is_valid(['beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden', 'beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden'], synth_data, 'beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden')
print('Beschikbaarheid medische check')
print(test_beschikbaarheid_medische.value_counts(), '\n')

test_beschikbaarheid_sociaal = zero_or_ge_is_valid(['beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie'], synth_data, 'beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie')
print('Beschikbaarheid sociaal check')
print(test_beschikbaarheid_sociaal.value_counts(), '\n')

test_beschikbaarheid_recent = zero_one_hot_encoding_is_valid(['beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden', 
                                                             'beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie'], synth_data)
print('Beschikbaarheid recent check')
print(test_beschikbaarheid_recent.value_counts(), '\n')

contacten_matches = {'contacten_onderwerp__arbeids_motivatie':'contacten_onderwerp_boolean__arbeids_motivatie',
                 'contacten_onderwerp__pre__intake':'contacten_onderwerp_boolean__pre__intake',
                 'contacten_onderwerp__werk_intake':'contacten_onderwerp_boolean__werk_intake',
                 'contacten_onderwerp_beoordelen_taaleis':'contacten_onderwerp_boolean_beoordelen_taaleis',
                 'contacten_onderwerp_contact_derden':'contacten_onderwerp_boolean_contact_derden',
                 'contacten_onderwerp_contact_met_aanbieder':'contacten_onderwerp_boolean_contact_met_aanbieder',
                 'contacten_onderwerp_diagnosegesprek':'contacten_onderwerp_boolean_diagnosegesprek',
                 'contacten_onderwerp_documenten__innemen_':'contacten_onderwerp_boolean_documenten__innemen_',
                 'contacten_onderwerp_documenttype__cv_':'contacten_onderwerp_boolean_documenttype__cv_',
                 'contacten_onderwerp_documenttype__overeenkomst_':'contacten_onderwerp_boolean_documenttype__overeenkomst_',
                 'contacten_onderwerp_financiële_situatie':'contacten_onderwerp_boolean_financiële_situatie',
                 'contacten_onderwerp_groepsbijeenkomst':'contacten_onderwerp_boolean_groepsbijeenkomst',
                 'contacten_onderwerp_inkomen':'contacten_onderwerp_boolean_inkomen',
                 'contacten_onderwerp_maatregel_overweging':'contacten_onderwerp_boolean_maatregel_overweging',
                 'contacten_onderwerp_matching':'contacten_onderwerp_boolean_matching',
                 'contacten_onderwerp_mutatie':'contacten_onderwerp_boolean_mutatie',
                 'contacten_onderwerp_no_show':'contacten_onderwerp_boolean_no_show',
                 'contacten_onderwerp_overige':'contacten_onderwerp_boolean_overige',
                 'contacten_onderwerp_overleg_met_inkomen':'contacten_onderwerp_boolean_overleg_met_inkomen',
                 'contacten_onderwerp_scholing':'contacten_onderwerp_boolean_scholing',
                 'contacten_onderwerp_terugbelverzoek':'contacten_onderwerp_boolean_terugbelverzoek',
                 'contacten_onderwerp_traject':'contacten_onderwerp_boolean_traject',
                 'contacten_onderwerp_uitnodiging':'contacten_onderwerp_boolean_uitnodiging',
                 'contacten_onderwerp_ziek__of_afmelding':'contacten_onderwerp_boolean_ziek__of_afmelding',
                 'contacten_onderwerp_zorg':'contacten_onderwerp_boolean_zorg'}
for key, value in contacten_matches.items():
    test_contacten = zero_or_ge_is_valid([value], synth_data, key)
    print(key, 'check')
    print(test_contacten.value_counts(), '\n')
    
test_kind = zero_or_ge_is_valid(['relatie_kind_heeft_kinderen'], synth_data, 'relatie_kind_huidige_aantal')
print('Kind check')
print(test_kind.value_counts(), '\n')

test_married = zero_or_ge_is_valid(['relatie_partner_huidige_partner___partner__gehuwd_'], synth_data, 'relatie_partner_aantal_partner___partner__gehuwd_')
print('Married check')
print(test_married.value_counts(), '\n')

test_pla_ondertekeningen = zero_or_ge_is_valid(['pla_ondertekeningen_actueel'], synth_data, 'pla_ondertekeningen_historie')
print('Pla ondertekeningen check')
print(test_pla_ondertekeningen.value_counts(), '\n')

ontheffing_test = zero_must_match_is_valid(ontheffing_vars, synth_data, 'ontheffing_hist_ind')
print('Ontheffing check')
print(ontheffing_test.value_counts(), '\n')

typering_test = zero_must_match_is_valid(typering_vars, synth_data, 'typering_hist_ind')
print('Typering check')
print(typering_test.value_counts(), '\n')

typering_hist_test = zero_or_ge_is_valid(['typering_hist_ind'], synth_data, 'typering_hist_aantal')
print('Typering History test')
print(typering_hist_test.value_counts(), '\n')
print(synth_data['persoonlijke_eigenschappen_spreektaal'].unique())

District check
True    12645
dtype: int64 

Neighborhood check
True    12645
dtype: int64 

Plaats check
True    12645
dtype: int64 

Onderdeel check
True     12508
False      137
dtype: int64 

Plaats check
True    12645
dtype: int64 

adres_recentste_wijk_noord check
True    12645
dtype: int64 

adres_recentste_wijk_feijenoord check
True    12645
dtype: int64 

adres_recentste_wijk_ijsselmonde check
True    12645
dtype: int64 

adres_recentste_wijk_delfshaven check
True    12645
dtype: int64 

Other district test
True    12645
dtype: int64 

Reading check
True    12645
dtype: int64 

Writing check
True    12645
dtype: int64 

Speaking check
True    12645
dtype: int64 

Aantal worden check
True    12645
dtype: int64 

Ingelvuld uniek check
True    12645
dtype: int64 

Beschikbaarheid huidig check
True    12645
dtype: int64 

Beschikbaarheid medische check
True    12645
dtype: int64 

Beschikbaarheid sociaal check
True    12645
dtype: int64 

Beschikbaarheid recent check
True    12645


In [382]:
###GENERATE EVALUATION REPORTS###
print(NewRowSynthesis.compute(
    real_data=real,
    synthetic_data=synth,
    metadata=meta_dic
))#should be 1.0 if it is fully anonymous
eval_col_shapes = report.get_details(property_name='Column Shapes') #compares cols between synth and real data
eval_col_pairs = report.get_details(property_name='Column Pair Trends') #compares col pairs between synth and real data
eval_col_shapes.to_csv('../data/01_raw/eval_col_shapes.csv')
eval_col_pairs.to_csv('../data/01_raw/eval_col_pairs.csv')

1.0
